In [1]:
using CSV, Dates, JuliaDB

In [2]:
# OPen the file and place its contents in an array name lines
function read_file(ionex_file)
    #print("Reading", ionex_file, "\n")
    file = open(ionex_file);
    lines = readlines(file)
    close(file)
    lines  # Julia automatically returns the last statement in a function
end

read_file (generic function with 1 method)

In [3]:
function get_map_values(lines)
    # This function could be improved by reading only the TEC map portion of the file
    #print("Inside get_map_values \n")
    rms_map = 0  # Some files do not have a complete rms map 
    index = 0
    tec_values = []   # Tec values per longitud, lat(-180,180)
    tec = []          # All the tec values in a file
    dates_array = []  # Store the different time stamps
    
    for line in lines
        index += 1
        
        # Obtain time_stamp and tec values for every map (13 TEC maps in a file)
        if occursin(r"START OF TEC MAP", line)
            date_line = lines[index+1]
            date_units = []
            string = split(date_line)
            
            for x in 1:6
                push!(date_units,parse(Int,string[x]))
            end
            
            # Date units 
            year = date_units[1]
            month = date_units[2]
            day = date_units[3]
            hour = date_units[4]
            minutes = date_units[5]
            seconds = date_units[6]
            
            # Map time_stamp for a period(2 hours)
            time_stamp = DateTime(year, month, day, hour, minutes, seconds)
            push!(dates_array, time_stamp)
        end
        
        if occursin(r"START OF RMS MAP", line)
            rms_map = 1
            #print("RMS MAP reached!\n")
            return dates_array, tec
        end
        
        # Tec values per latitud in the longitud range (-180,180)
        if occursin(r"LAT/LON1/LON2/DLON/H", line) && (rms_map == 0)
            for x in 1:5
                tec_values = split(lines[index + x])
                for n in tec_values
                    #if occursin(r"^-?[0-9]*$", n)
                    if occursin(r"^-?\d*\.{0,1}\d+$", n)
                        push!(tec, n)
                    else
                        push!(tec, 0)
                        print("Issue found in ", dates_array[1], n, "\n")
                    end
                end
            end
        end
        
    end
    return dates_array, tec
end

get_map_values (generic function with 1 method)

In [4]:
# used to create a latitud, to hold 2-hour-interval Tec map data
function create_latitud(start, stop, step)
#    print("Inside create_latitud function \n")
    coordinates = []
    for i in range(start, stop=stop, step=step)
        for j in 1:73
            push!(coordinates,i)
        end
    end
    coordinates # Julia automatically returns the last statement
end
    

create_latitud (generic function with 1 method)

In [5]:
function get_coordinates()
#    print("Inside get_coordinates\n")
    latitud = create_latitud(87.5, -87.5, -2.5)
    longitud = collect(Iterators.flatten([collect(-180:5:180) for i in 1:71])) # I Still don't grok this!
    return latitud, longitud
end

get_coordinates (generic function with 1 method)

In [6]:
# Create CSV for daily tec map data.
function create_csv(directory, ionex_file)
   # print("Inside create_csv\n")
    lines = read_file(string(directory, ionex_file))
    dates_array, tec = get_map_values(lines)
    latitud, longitud = get_coordinates()
    
    csv_file = string("../tec_csv_files/", ionex_file, ".csv")
    table_file = string("../tec_tables/", ionex_file, "_table")
    
    start = 1
    map_tec_values = 5183  # Total tec values in a 2 hour map period
    stop = map_tec_values
    #print("Starting to arrange data for CSV\n")
    for d in dates_array
        #print("\nStart ", start)
        time_stamp = [d for i in 1:map_tec_values]
        #print("Looking at TEC value\n")
        tec_value = [parse(Int,value) for value in tec[start:stop]]
        t = table((time_stamp=time_stamp, latitud=latitud, longitud=longitud, tec_value=tec_value); pkey = [:time_stamp, :latitud, :longitud, :tec_value])
        
        CSV.write(csv_file ,t, append=true)
        JuliaDB.save(t, table_file)
        
        start += map_tec_values
        stop += map_tec_values
    end
end


create_csv (generic function with 1 method)

## Make sure the downloaded files are uncompressed by running the command "uncompress *.Z" from within the folder containg the files
You should end up with all files ending in "i" instead of ".Z"

Some files were smaller than the average 700k, so files smaller than 500k were removed from the processing.  These are held in a TempDir folder.

In [7]:
function create_database()
    counter = 0
    directory = "../ionex_files/"
    for f in files
        print(counter, " ", f, "\n")
        if isfile(string("../tec_csv_files/", f, ".csv")) == false
            print("Writting file:", f, "\n")
            create_csv(directory, f)
        end
        counter += 1
    end
end

create_database (generic function with 1 method)

In [10]:
# Read the contents of the directory in order to process the files to create the csv 
files = readdir("/home/antonio/Repos/iono2/ionex_files")
@time create_database()

0 jplg0010.00i
1 jplg0010.01i
2 jplg0010.02i
3 jplg0010.03i
4 jplg0010.04i
5 jplg0010.05i
6 jplg0010.06i
7 jplg0010.07i
8 jplg0010.08i
9 jplg0010.09i
10 jplg0010.10i
11 jplg0010.11i
12 jplg0010.12i
13 jplg0010.13i
14 jplg0010.14i
15 jplg0010.15i
16 jplg0010.16i
17 jplg0010.17i
18 jplg0010.18i
19 jplg0010.99i
20 jplg0020.00i
21 jplg0020.01i
22 jplg0020.02i
23 jplg0020.03i
24 jplg0020.04i
25 jplg0020.05i
26 jplg0020.06i
27 jplg0020.07i
28 jplg0020.08i
29 jplg0020.09i
30 jplg0020.10i
31 jplg0020.11i
32 jplg0020.12i
33 jplg0020.13i
34 jplg0020.14i
35 jplg0020.15i
36 jplg0020.16i
37 jplg0020.17i
38 jplg0020.18i
39 jplg0020.99i
40 jplg0030.00i
41 jplg0030.01i
42 jplg0030.02i
43 jplg0030.03i
44 jplg0030.04i
45 jplg0030.05i
46 jplg0030.06i
47 jplg0030.07i
48 jplg0030.08i
49 jplg0030.09i
50 jplg0030.10i
51 jplg0030.11i
52 jplg0030.12i
53 jplg0030.13i
54 jplg0030.14i
55 jplg0030.15i
56 jplg0030.16i
57 jplg0030.17i
58 jplg0030.18i
59 jplg0030.99i
60 jplg0040.00i
61 jplg0040.01i
62 jplg0040.02i
63

## File jplg0430.00i from JPL has dirty data "***" showing up in strings meant to be numbers
File will be replaced by its igsg equivalent

## NOTE data will be processed per earthquake time, looking back through all the available data but just using the data to calculate the eartquake, so the data from the other months from when the earthquake happened
lets say oct 1 2000, all the years covering 20 days before oct 1 will be the rows in the observations to predict the magnitud of the eartquake.

## Loading 15GB CSV file, JuliaDB